In [18]:
import os
import json
from pathlib import Path
from llama_index.llms.ollama import Ollama
from llama_index.core import VectorStoreIndex
from llama_index.core.embeddings import resolve_embed_model
from llama_index.readers.json import JSONReader
from llama_index.core.node_parser import JSONNodeParser
from llama_index.readers.file import FlatReader

In [19]:
from sqlalchemy import make_url
from llama_index.vector_stores.postgres import PGVectorStore

db_name = "vector_db"
host = "localhost"
password = "password"
port = "5432"
user = "lavo" #<-------------------------------------------- change this to your username !!!

vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
    table_name="json_data",
    embed_dim=1024,  # openai embedding dimension
)

In [20]:
# parser = JSONNodeParser()     # if we want to split the documents into nodes
llm = Ollama(model="llama3.2", request_timeout=180.0) 

# retrieving from existing db (if we already have the embeddings)

In [21]:
vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
    table_name="json_data",
    embed_dim=1024,  # openai embedding dimension
)

In [22]:
from llama_index.core.schema import NodeWithScore
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List


class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [23]:
embed_model = resolve_embed_model("local:BAAI/bge-m3")

In [24]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=14
)

# VectorStoreQuery is defined using this query embedding

In [25]:
query_str = "General Summarized Overview Large Capacity Cutting Machine 2?"

query_embedding = embed_model.get_query_embedding(query_str)

In [26]:
# construct vector store query
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

In [27]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

In [28]:
query_str = "Hello"

## to run this i need a VectorStoreQuery.. we need to explain what is it

In [29]:
response = query_engine.query(query_str)

In [30]:
print(response)

It seems like you're greeting. How can I assist you today?


# API test

In [ ]:
import requests

<Response [403]>
{'detail': 'Not authenticated'}


In [35]:
url_user = 'https://api-656930476914.europe-west1.run.app/api/v1.0/user/'

# Make the GET request
response = requests.get(url_user)

data = response.json()
print(response)
print(data)

<Response [403]>
{'detail': 'Not authenticated'}


In [ ]:
from requests.auth import HTTPBasicAuth
# bho non va ma se li metto manualmente nell'url funziona.
user = "ffm@example.com"
password = "passwordffm"
#response = requests.get(url, auth=HTTPBasicAuth(user, password))


In [55]:
url_login = 'https://api-656930476914.europe-west1.run.app/api/v1.0/user/login?email=ffm%40example.com&password=passwordffm'

# Make the POST request
response = requests.post(url_login)

data = response.json()
print(response)


<Response [200]>


In [56]:
token = data['id_token']

In [58]:
url = 'https://api-656930476914.europe-west1.run.app/api/v1.0/kpi/'

headers = {
    'Authorization': f'Bearer {token}'
}

# Make the GET request
response = requests.get(url, headers=headers)

data = response.json()
print(response)
print(data)

<Response [200]>
[{'_id': '673a6ad2d9e0b151b88cbed0', 'name': 'working_time', 'type': 'placeholder', 'description': 'placeholder', 'unite_of_measure': 'placeholder'}, {'_id': '673a6ad2d9e0b151b88cbed1', 'name': 'idle_time', 'type': 'placeholder', 'description': 'placeholder', 'unite_of_measure': 'placeholder'}, {'_id': '673a6ad3d9e0b151b88cbed2', 'name': 'offline_time', 'type': 'placeholder', 'description': 'placeholder', 'unite_of_measure': 'placeholder'}, {'_id': '673a6ad3d9e0b151b88cbed3', 'name': 'consumption', 'type': 'placeholder', 'description': 'placeholder', 'unite_of_measure': 'placeholder'}, {'_id': '673a6ad4d9e0b151b88cbed4', 'name': 'power', 'type': 'placeholder', 'description': 'placeholder', 'unite_of_measure': 'placeholder'}, {'_id': '673a6ad4d9e0b151b88cbed5', 'name': 'consumption_working', 'type': 'placeholder', 'description': 'placeholder', 'unite_of_measure': 'placeholder'}, {'_id': '673a6ad4d9e0b151b88cbed6', 'name': 'consumption_idle', 'type': 'placeholder', 'desc

In [59]:
data

[{'_id': '673a6ad2d9e0b151b88cbed0',
  'name': 'working_time',
  'type': 'placeholder',
  'description': 'placeholder',
  'unite_of_measure': 'placeholder'},
 {'_id': '673a6ad2d9e0b151b88cbed1',
  'name': 'idle_time',
  'type': 'placeholder',
  'description': 'placeholder',
  'unite_of_measure': 'placeholder'},
 {'_id': '673a6ad3d9e0b151b88cbed2',
  'name': 'offline_time',
  'type': 'placeholder',
  'description': 'placeholder',
  'unite_of_measure': 'placeholder'},
 {'_id': '673a6ad3d9e0b151b88cbed3',
  'name': 'consumption',
  'type': 'placeholder',
  'description': 'placeholder',
  'unite_of_measure': 'placeholder'},
 {'_id': '673a6ad4d9e0b151b88cbed4',
  'name': 'power',
  'type': 'placeholder',
  'description': 'placeholder',
  'unite_of_measure': 'placeholder'},
 {'_id': '673a6ad4d9e0b151b88cbed5',
  'name': 'consumption_working',
  'type': 'placeholder',
  'description': 'placeholder',
  'unite_of_measure': 'placeholder'},
 {'_id': '673a6ad4d9e0b151b88cbed6',
  'name': 'consumpt